In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebram
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from wordcloud import WordCloud
import pickle
test_1 = pd.read_csv("../input/quora1/test.csv")
train_1=pd.read_csv("../input/quora1/train.csv")
train_c=train_1["question_text"]
train_c_y=train_1["target"]
test_c=test_1["question_text"]
sample_train=list(train_c)
sample_test=list(test_c)
sub=pd.read_csv('../input/quora1/sample_submission.csv')
sub.drop(["target"],axis=1)
y=train_1['target'].values

In [ ]:
import pickle
with open("../input/dataset/embedding.txt", "rb") as fp:   # Unpickling
    embeddings = pickle.load(fp)
    
    
with open("../input/dataset/list_labels.txt", "rb") as fp:   # Unpickling
    list_labels = pickle.load(fp)    
    


In [ ]:
Y=train_c_y.to_list()
type(Y)

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(train_1, test_size = 0.2, random_state = 1)
train_set_q=train_set["question_text"]
test_set_q=test_set["question_text"]


In [ ]:
X_train_c = train_set.iloc[:,1:2].values
Y_train_c = train_set.iloc[:,2].values
X_test_c = test_set.iloc[:,1:2].values
Y_test_c = test_set.iloc[:,2].values


In [ ]:
X_train = train_1.iloc[:,1].values
Y_train = train_1.iloc[:,2].values
X_test = test_1.iloc[:,1].values
type(X_train)


In [ ]:
print("Number of train data points:",train_1.shape[0])
print("Number of test data points:",test_1.shape[0])
print("Shape of Train Data:", train_1.shape)
print("Shape of Test Data:", test_1.shape)
test_1.values

In [ ]:
print("For testing Number of train data points:",train_set.shape[0])
print("For testing Number of test data points:",test_set.shape[0])
print("For testing Shape of Train Data:", train_set.shape)
print("For testing Shape of Test Data:", test_set.shape)
train_set.head()

In [ ]:
# Total sincere ans insincere questions
fig_dims = (24, 12)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(train_1['target'])

In [ ]:
print("Percetage of insincere comments: ", round(train_1["target"].mean()*100, 2))
print("Percetage of sincere comments: ", 100-round(train_1["target"].mean()*100, 2))

In [ ]:
import string
def remove_newline(text):
    """
    remove \n and  \t
    """
    text = re.sub('\n', ' ', text)
    text = re.sub('\t', ' ', text)
    text = re.sub('\b', ' ', text)
    text = re.sub('\r', ' ', text)
    return text


def spacing_punctuation(text):
    """
    add space before and after punctuation and symbols
    """
    re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤$&#‘’])')
    return re_tok.sub(r' \1 ', text)


def remove_punctuation(text):
    """
    remove punctuation from text
    """
    re_tok = re.compile(f'([{string.punctuation}])')
    return re_tok.sub(' ', text)


def spacing_number(text):
    """
    add space before and after numbers
    """
    re_tok = re.compile('([0-9]{1,})')
    return re_tok.sub(r' \1 ', text)


def decontracted(text):
    """
    de-contract the contraction
    """
    # specific
    text = re.sub(r"(W|w)on\'t", "will not", text)
    text = re.sub(r"(C|c)an\'t", "can not", text)

    # general
    text = re.sub(r"(I|i)\'m", "i am", text)
    text = re.sub(r"(A|a)in\'t", "is not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    return text


def clean_number(text):
    """
    replace number with hash
    """
    text = re.sub('[0-9]{5,}', '#####', text)
    text = re.sub('[0-9]{4}', '####', text)
    text = re.sub('[0-9]{3}', '###', text)
    text = re.sub('[0-9]{2}', '##', text)
    return text


def remove_number(text):
    """
    numbers are not toxic
    """
    return re.sub('\d+', ' ', text)


def remove_space(text):
    """
    remove extra spaces and ending space if any
    """
    text = re.sub('\s+', ' ', text)
    text = re.sub('\s+$', '', text)
    return text
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext


def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext

In [ ]:
#Method used for cleaning
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"


stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    text = spacing_punctuation(text)
    text = spacing_number(text)
    text = decontracted(text)
    text = remove_number(text)
    text=cleanHtml(text)
    text = remove_space(text)
    text = remove_punctuation(text)
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens).strip()

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

def text_cleaning(text):
    text = clean_text(text)
    text = clean_numbers(text)
    text = replace_typical_misspell(text)
    return text



In [ ]:
train_1["question_text"]=train_c.apply(lambda x:preprocess(x,stem=True))



In [ ]:
train_1["question_text"] = train_1["question_text"].apply(text_cleaning)

In [ ]:
train_1.head(20)

In [ ]:
tweets=train_1["question_text"].values.tolist()
type(tweets)

In [ ]:
df_train_list=list(train_1["question_text"])
type(df_train_list)

#Tokenization

import nltk
  
from nltk.tokenize import word_tokenize 
token_train=[]
for word in df_train_list:
    x=word_tokenize(word)
    token_train.append(x)


token_train

In [ ]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

train_1["tokens"] = train_1["question_text"].apply(tokenizer.tokenize)
train_1.head()

In [ ]:
type(train_1)

In [ ]:
list_labels=train_1["target"].tolist()

In [ ]:
type(list_labels)

In [ ]:
import gensim

word2vec_path = "../input/worl2vec/GoogleNews-vectors-negative300.bin"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [ ]:
word2vec["god"].shape

In [ ]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_questions, generate_missing=False):
    embeddings = clean_questions['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [ ]:
embeddings = get_word2vec_embeddings(word2vec, train_1)

In [ ]:
Data_word=np.array(embeddings)

In [ ]:
np.savetxt("Data_word.txt", Data_word) 

In [ ]:
import pickle
with open("embedding.txt", "wb") as fp:   #Pickling
    pickle.dump(embeddings, fp)

In [ ]:
import pickle
with open("list_labels.txt","wb") as fp:
    pickle.dump(list_labels,fp)

In [ ]:
# For Word2vec
from sklearn.model_selection import train_test_split
X_train_word2vec, X_test_word2vec, y_train_word2vec, y_test_word2vec = train_test_split(embeddings, list_labels, 
                                                                                        test_size=0.2, random_state=40)

In [ ]:

#For tf idf split dataset into train and test

x_train, x_test,y_train,y_test = train_test_split(nb_tl, y,test_size=0.2, random_state=40)


In [ ]:
type(y_train_word2vec)

In [ ]:
type(X_train_word2vec)

In [ ]:
y=train_1['target'].values
type(y)


In [ ]:
X_train_W_np=np.array(X_train_word2vec)
y_train_W_np=np.array(y_train_word2vec)
X_test_W_np=np.array(X_test_word2vec)
y_test_W_np=np.array(y_test_word2vec)

from scipy import sparse
X_train_W_cs=sparse.csr_matrix(X_train_word2vec)
y_train_W_cs=np.array(y_train_word2vec)
X_test_W_ncs=sparse.csr_matrix(X_test_word2vec)

'''
https://dl.acm.org/doi/epdf/10.5555/2390665.2390688
https://marvinlsj.github.io/2018/11/23/NBSVM%20for%20sentiment%20and%20topic%20classification/

Logistic regression with NB log count ratio
'''
from scipy.sparse import csr_matrix,save_npz,load_npz
p = 1 + X_train_np[y_train_np==1].sum(0)
q = 1 + X_train_np[y_train_np==0].sum(0)
r = csr_matrix(np.log((p/(1 + (y_train_np==1).sum()))/(q/(1+ (y_train_np==0).sum()))))

def multiply(X,r):
    return X.dot(r)


# obj= NBTransformer().fit(train,y)
nb_train=multiply(X_train_np,r)
nb_test=multiply(X_test_np,r)

In [ ]:
# Scatter graph On Word2Vec
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib
import matplotlib.patches as mpatches


def plot_LSA(test_data, test_labels, savepath="PCA_demo.csv", plot=True):
        lsa = TruncatedSVD(n_components=2)
        lsa.fit(test_data)
        
        lsa_scores = lsa.transform(test_data)
        # color_mapper = {label:idx for idx,label in enumerate(set(test_labels))}
        # color_column = [color_mapper[label] for label in test_labels]
        print(set(test_labels))
        colors = ['orange','blue','black']
        if plot:
            plt.scatter(lsa_scores[:,0], lsa_scores[:,1], s=8, alpha=.8, c=test_labels, cmap=matplotlib.colors.ListedColormap(colors))
            red_patch = mpatches.Patch(color='orange', label='Zero ')
            green_patch = mpatches.Patch(color='black', label='One')
            plt.legend(handles=[red_patch, green_patch], prop={'size': 30})


fig = plt.figure(figsize=(16, 16))          
plot_LSA(embeddings, list_labels)
plt.show()

In [ ]:
# #Scatter graph On TF-idf
fig = plt.figure(figsize=(16, 16))          
plot_LSA(nb_tl, y)
plt.show()

In [ ]:
# Word2vec Logestic
clf_w2v = LogisticRegression(C=11.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', random_state=40)
clf_w2v.fit(X_train_word2vec, y_train_word2vec)
y_predicted_word2vec_demo = clf_w2v.predict(X_test_word2vec)

In [ ]:
# for Tf idf
clf_w2v = LogisticRegression(C=11.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', random_state=40)
clf_w2v.fit(x_train, y_train)
y_predicted_demo = clf_w2v.predict(x_test)

In [ ]:
type(y_predicted_word2vec_demo)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1


In [ ]:
#Word2vec Score
accuracy_word2vec, precision_word2vec, recall_word2vec, f1_word2vec = get_metrics(y_test_word2vec, y_predicted_word2vec_demo)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_word2vec, precision_word2vec, 
                                                                       recall_word2vec, f1_word2vec))

In [ ]:
import numpy as np
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.winter):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, fontsize=20)
    plt.yticks(tick_marks, classes, fontsize=20)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", 
                 color="white" if cm[i, j] < thresh else "black", fontsize=40)
    
    plt.tight_layout()
    plt.ylabel('True label', fontsize=30)
    plt.xlabel('Predicted label', fontsize=30)

    return plt

In [ ]:
#Word2Vec embedding Confussion Matrix
cm_w2v = confusion_matrix(y_test_word2vec, y_predicted_word2vec_demo)
fig = plt.figure(figsize=(10, 10))
plot = plot_confusion_matrix(cm_w2v, classes=['ZERO','ONE'], normalize=False, title='Confusion matrix')
plt.show()
print("Word2Vec confusion matrix")
print(cm_w2v)


In [ ]:
#Tf idf Score
accuracy_tfidf, precision_tfidf, recall_tfidf, f1_tfidf = get_metrics(y_test, y_predicted_demo)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_tfidf, precision_tfidf, 
                                                                       recall_tfidf, f1_tfidf))

In [ ]:
#Tf idf embedding Confussion Matrix
cm_tf = confusion_matrix(y_test, y_predicted_demo)
fig = plt.figure(figsize=(10, 10))
plot = plot_confusion_matrix(cm_tf, classes=['ZERO','ONE'], normalize=False, title='Confusion matrix')
plt.show()
print("Tf-idf confusion matrix")
print(cm_tf)


In [ ]:
#preprocessing
test_1["question_text"]=test_c.apply(lambda x:preprocess(x,stem=True))
test_1["question_text"] = test_1["question_text"].apply(text_cleaning)

In [ ]:
#tokanize
test_1["tokens"] = test_1["question_text"].apply(tokenizer.tokenize)

In [ ]:
#embedding
embeddings_1 = get_word2vec_embeddings(word2vec, test_1)


In [ ]:
clf_w2v.fit(embeddings, list_labels)
y_predicted_word2vec = clf_w2v.predict(embeddings_1)

In [ ]:
type(y_predicted_word2vec)

In [ ]:
submission=pd.read_csv('../input/quora1/sample_submission.csv')
submission.drop(["target"],axis=1)
submission['target']=y_predicted_word2vec
submission.to_csv('y_predicted_word2vec.csv',index=False)

In [ ]:
#### END

In [ ]:
! pip install catboost
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_breast_cancer 
from sklearn.svm import SVC 
  


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from scipy.sparse import csr_matrix,save_npz,load_npz
nb_tl=load_npz('../input/test-and-train-data/nb_train.npz')
nb_testl=load_npz('../input/test-and-train-data/nb_test.npz')

In [ ]:
dic={'question':nb_tl,'target':Y}

In [ ]:
train_N=pd.DataFrame(dic)

In [ ]:
train_N1=train_N.drop(['target'],axis=1)

In [ ]:
#split dataset into train and test

x_train, x_test,y_train,y_test = train_test_split(nb_tl, y,test_size=0.2, random_state=40)


In [ ]:
type(Y)

In [ ]:
type(x_train)

In [ ]:
type(y_train)

# LogisticRegression

In [ ]:
model = LogisticRegression(random_state=1)
model.fit(x_train, y_train)
model.score(x_test, y_test)

# LogisticRegression on word to vec

In [ ]:
model = LogisticRegression(random_state=1)
model.fit(X_train_word2vec, y_train_word2vec)
model.score(X_test_word2vec, y_test_word2vec)

# Hyperparameter tuning on LogisticRegression

In [ ]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }

logreg_cv=GridSearchCV(cv=None,
             estimator=LogisticRegression(C=1.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]})

logreg_cv.fit(nb_tl,y)

* ****Best Score**:**  0.9548472379449044
* ****Best Params**: ** {'C': 1}

# SVM

model = svm.SVC()
model.fit(x_train, y_train)
model.score(x_test, y_test)

# SVM on wordtovec
model = svm.SVC(kernel='linear')
model.fit(X_train_word2vec, y_train_word2vec)
model.score(X_test_word2vec, y_test_word2vec)

## On SVC DID Hyperparameter tuning 

#LinearScv
param_grid = {
    'C':            [10,40,80],
    'kernel':       ['linear'],                   # precomputed,'poly', 'sigmoid'
    'degree':       [3,4],
    'gamma':        [0.001,0.0001],
    'shrinking':    [True],
    'probability':  [False],
    'cache_size':   [2000],
    'class_weight': [None],
    'verbose':      [False],
    'max_iter':     [-1],
    'random_state': [None],
    }
  
grid = RandomizedSearchCV(SVC(), param_grid, refit = True, verbose = 10) 
  
# fitting the model for grid search 
grid.fit(nb_tl, y)
print('Best Score: ', grid.best_score_)
print('Best Params: ', grid.best_params_)

# DecisionTree

model = tree.DecisionTreeClassifier(random_state=1)
model.fit(x_train, y_train)
model.score(x_test, y_test)

# DecisionTreeTO WORDTOVEC
model = tree.DecisionTreeClassifier(random_state=1)
model.fit(X_train_W_np, y_train_W_np)
model.score(X_test_W_np, y_test_W_np)


# Hyperparameter tuning on RandomForestClassifie

In [ ]:
rfc = RandomForestClassifier()

n_estimators = [250,300,350,400]
max_depth = [6,7,8]
param = dict(n_estimators=n_estimators,
                  max_depth=max_depth)

random = RandomizedSearchCV(estimator=rfc,
                            param_distributions=param,
                            scoring='roc_auc',
                            verbose=100, n_jobs=-1,
                            n_iter=1000)
random_result = random.fit(nb_tl, y)

print('Best Score: ', random_result.best_score_)
print('Best Params: ', random_result.best_params_)

* **Best Score:  0.9139219026051947
* **Best Params:  {'n_estimators': 400, 'max_depth': 7}

In [ ]:
modelr= RandomForestClassifier(n_estimators=400,max_depth=7)
modelr.fit(x_train, y_train)
modelr.score(x_test,y_test)

In [ ]:
# RandomForestClassifier WORDTOVEC
modelr1= RandomForestClassifier(n_estimators=400,max_depth=7)
modelr1.fit(X_train_word2vec, y_train_word2vec)
modelr1.score(X_test_word2vec, y_test_word2vec)


In [ ]:
model= RandomForestClassifier(n_estimators=400,max_depth=7)
model.fit(nb_tl, y)




In [ ]:
filename="model_random"
pickle.dump(model,open(filename,'wb'))
model_1 = pickle.load(open(filename, 'rb'))

In [ ]:
pred_ram=model.predict(nb_testl)

In [ ]:

sub1=pd.read_csv('../input/quora1/sample_submission.csv')
sub1.drop(["target"],axis=1)
sub1['target']=pred_ram
sub1.to_csv('ran_pred1.csv',index=False)

# XGBClassifier

In [ ]:
modelx=xgb.XGBClassifier(random_state=1,learning_rate=0.15,n_estimators=700)
modelx.fit(x_train, y_train)
modelx.score(x_test,y_test)

In [ ]:
modelx1=xgb.XGBClassifier(random_state=1,learning_rate=0.15,n_estimators=700)
modelx1.fit(X_train_word2vec, y_train_word2vec)
modelx1.score(X_test_word2vec, y_test_word2vec)

In [ ]:
model=xgb.XGBClassifier(random_state=1,learning_rate=0.25,n_estimators=1400)
model.fit(nb_tl, y)



In [ ]:
filename = 'XG_model.sav'
pickle.dump(model, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))


In [ ]:
pred_xg=model.predict(nb_testl)


In [ ]:
sub=pd.read_csv('../input/quora/sample_submission.csv')


In [ ]:
sub.drop(["target"],axis=1)
sub['target']=pred_xg

In [ ]:
sub.to_csv('xg_pred',index=False)

# Hyperparameter tuning

In [ ]:
#LinearScv
param_grid = {'C': [1,100],  
              'gamma': [0.1, 0.01], 
              'kernel': ['linear'],
               'degree':[3]}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 10) 
  
# fitting the model for grid search 
grid.fit(nb_tl, y)

In [ ]:
pred_svc=grid.predict(nb_testl)

In [ ]:
sub=pd.read_csv('../input/quora/sample_submission.csv')
sub.drop(["target"],axis=1)
sub['target']=pred_svc

In [ ]:
sub.to_csv('pred_svc',index=False)

In [ ]:
print('Best Score: ', grid.best_score_)
print('Best Params: ', grid.best_params_)

1. # Random Search ON RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()

n_estimators = [350,400,550,600]
max_depth = [5,6,7]
param = dict(n_estimators=n_estimators,
                  max_depth=max_depth)

random = RandomizedSearchCV(estimator=rfc,
                            param_distributions=param,
                            scoring='roc_auc',
                            verbose=1000, n_jobs=-1,
                            n_iter=1000)
random_result = random.fit(x_train, y_train)

print('Best Score: ', random_result.best_score_)
print('Best Params: ', random_result.best_params_)

In [ ]:
#Random Search WORDTOVEC
rfc = RandomForestClassifier()

n_estimators = [350,400,550,600]
max_depth = [6,7,8,9]
param = dict(n_estimators=n_estimators,
                  max_depth=max_depth)

random = RandomizedSearchCV(estimator=rfc,
                            param_distributions=param,
                            scoring='roc_auc',
                            verbose=100, n_jobs=-1,
                            n_iter=1000)
random_result = random.fit(X_train_W_np, y_train_W_np)

print('Best Score: ', random_result.best_score_)
print('Best Params: ', random_result.best_params_)


In [ ]:
rfc = RandomForestClassifier()

n_estimators = [200,400,500,600]
max_depth = [3,4,7,8,9,10,11,16]
param = dict(n_estimators=n_estimators,
                  max_depth=max_depth)

random = RandomizedSearchCV(estimator=rfc,
                            param_distributions=param,
                            scoring='roc_auc',
                            verbose=1000, n_jobs=-1,
                            n_iter=1000)
random_result = random.fit(nb_tl, y)

print('Best Score: ', random_result.best_score_)
print('Best Params: ', random_result.best_params_)

In [ ]:
filename = 'ramdomforestbest.sav'
pickle.dump(random, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
ramdomforset=random.predict(nb_testl)
sub=pd.read_csv('../input/quora1/sample_submission.csv')
sub.drop(["target"],axis=1)
sub['target']=pred_ram
sub.to_csv('ramdomforset',index=False)



# MaxVoting

In [ ]:

model1 = LogisticRegression(C=11.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001)
model2 =  RandomForestClassifier(max_depth=7, n_estimators=400,random_state=0)
model3 = SGDClassifier(random_state=1,alpha=0.05, loss="modified_huber", max_iter=1000, penalty="l1")
model11 = VotingClassifier(estimators=[('lr', model1), ('dt', model2), ('NB', model3)])
model11.fit(x_train,y_train)
model11.score(x_test,y_test)

In [ ]:
# MAx Voting wordtovec

model1 = LogisticRegression(C=11.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001)
model2 =  RandomForestClassifier(max_depth=7, n_estimators=400,random_state=0)
model3 = SGDClassifier(random_state=1,alpha=0.05, loss="modified_huber", max_iter=1000, penalty="l1")
model12 = VotingClassifier(estimators=[('lr', model1), ('dt', model2), ('NB', model3)])
model12.fit(X_train_word2vec, y_train_word2vec)
model12.score(X_test_word2vec, y_test_word2vec)

In [ ]:
# On Main test data we apply ramdom forest
model1 = LogisticRegression(C=11.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001)
model2 =  RandomForestClassifier(max_depth=7, n_estimators=400,random_state=0)
model3 = SGDClassifier(random_state=1,alpha=0.05, loss="modified_huber", max_iter=1000, penalty="l1")


model_m = VotingClassifier(estimators=[('lr', model1), ('rf', model2), ('NB', model3)])
model_m.fit(nb_tl, y)

In [ ]:
maxvoting=model_m.predict(nb_testl)
submission['target']=maxvoting
submission.to_csv('maxvoting.csv',index=False)

# Weighted Averaging

In [ ]:
model1 = LogisticRegression(C=11.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001)
model2 =  RandomForestClassifier(max_depth=7, n_estimators=400,random_state=0)
model3 = SGDClassifier(random_state=1,alpha=0.05, loss="modified_huber", max_iter=1000, penalty="l1")

model1.fit(x_train,y_train)
model2.fit(x_train,y_train)
model3.fit(x_train,y_train)

pred1 = model1.predict_proba(x_test)
pred2 = model2.predict_proba(x_test)
pred3 = model3.predict_proba(x_test)

weighted_prediction = (0.7*pred1)+(0.2*pred2)+(0.1*pred3)
labelprediction = np.argmax(weighted_prediction, axis = 1)

accuracy_score(labelprediction, y_test)

In [ ]:
# Weighted Averaging wordtovec
model1 = LogisticRegression(C=11.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001)
model2 =  RandomForestClassifier(max_depth=7, n_estimators=400,random_state=0)
model3 = SGDClassifier(random_state=1,alpha=0.05, loss="modified_huber", max_iter=1000, penalty="l1")

model1.fit(X_train_word2vec, y_train_word2vec)
model2.fit(X_train_word2vec, y_train_word2vec)
model3.fit(X_train_word2vec, y_train_word2vec)

pred1 = model1.predict_proba(X_test_word2vec)
pred2 = model2.predict_proba(X_test_word2vec)
pred3 = model3.predict_proba(X_test_word2vec)

weighted_prediction = (0.7*pred1)+(0.2*pred2)+(0.1*pred3)
labelprediction1 = np.argmax(weighted_prediction, axis = 1)

accuracy_score(labelprediction1,y_test_word2vec)

In [ ]:
# # Weighted Averaging on main test data

model1 = LogisticRegression(C=11.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001)
model2 =  RandomForestClassifier(max_depth=7, n_estimators=400,random_state=0)
model3 = SGDClassifier(random_state=1,alpha=0.05, loss="modified_huber", max_iter=1000, penalty="l1")

model1.fit(nb_tl, y)
model2.fit(nb_tl, y)
model3.fit(nb_tl, y)

pred1 = model1.predict_proba(nb_testl)
pred2 = model2.predict_proba(nb_testl)
pred3 = model3.predict_proba(nb_testl)

weighted_prediction = (0.7*pred1)+(0.2*pred2)+(0.1*pred3)
weightedavg = np.argmax(weighted_prediction, axis = 1)

In [ ]:
submission['target']=weightedavg
submission.to_csv('weightedavg.csv',index=False)

# Stacking

In [ ]:
model1 = LogisticRegression(C=11.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001)
model2 =  RandomForestClassifier(max_depth=7, n_estimators=400,random_state=0)
model3 = SGDClassifier(random_state=1,alpha=0.05, loss="modified_huber", max_iter=1000, penalty="l1")


stack = StackingClassifier(estimators=[('dt',model1),('lr',model2)], final_estimator=model2,cv=10)
stack.fit(nb_tl,y)
# stack.score(x_test,y_test)

In [ ]:
Stacking=stack.predict(nb_testl)
submission['target']=Stacking
submission.to_csv('Stacking.csv',index=False)

# Blending

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(nb_t1,y, test_size=0.2, random_state=0)

In [ ]:
# train, test = train_test_split(train_N, test_size=0.2, random_state=0)

# x_train=train.drop(['target'], axis=1)
# y_train=train['target']

# x_val=test.drop(['target'], axis=1)
# y_val=test['target']

# x_val = x_val.reset_index(drop = True)
# x_test = x_test.reset_index(drop = True)

model1 = SGDClassifier(random_state=1, loss="modified_huber", alpha=0.05, max_iter=1000, penalty="l1")
model1.fit(x_train, y_train)

val_pred1=model1.predict(x_test)
test_pred1=model1.predict(nb_testl)

val_pred1=pd.DataFrame(val_pred1)
test_pred1=pd.DataFrame(test_pred1)

model2 =  RandomForestClassifier(max_depth=7, n_estimators=400,random_state=0)
model2.fit(x_train,y_train)

val_pred2=model2.predict(x_test)
test_pred2=model2.predict(nb_testl)

val_pred2=pd.DataFrame(val_pred2)
test_pred2=pd.DataFrame(test_pred2)

model3 = LogisticRegression(C=11.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001)
model3.fit(x_train,y_train)

val_pred3=model3.predict(x_test)
test_pred3=model3.predict(nb_testl)

val_pred3=pd.DataFrame(val_pred3)
test_pred3=pd.DataFrame(test_pred3)

In [ ]:
from scipy.sparse import hstack
df_val=hstack((x_test, val_pred1,val_pred2,val_pred3))
df_test = hstack((nb_testl, test_pred1,test_pred2,test_pred3))

In [ ]:
model = LogisticRegression(C=11.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001)
model.fit(df_val,y_test)

In [ ]:
blending=model.predict(df_test)
submission['target']=blending
submission.to_csv('blending.csv',index=False)

In [ ]:
#stop word removal
  
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stopw=set(stopwords.words('english'))
arr0=[]
for i in ls1:  
    filtered_sentence = [w for w in i if not w in stopw] 
    arr0.append(filtered_sentence)
arr=[]                
bad_word=['~','`','!','@','#','$','%','^','&','*','(',')','_','-','=','+','|',';',':','<','>','.','/','?','[',']','{','}']
for y in arr0:
    lss=[q for q in y if not q in bad_word]
    arr.append(lss)

In [ ]:
kaggle competitions submit -c quora -f submission.csv -m "Message"

In [ ]:
#stemming\lemmataizer
import nltk
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def get_wordnet_pos(word2):
    pos=nltk.pos_tag([word2])[0][1][0].upper()
    word_pos={
             "J": wordnet.ADJ,
             "N": wordnet.NOUN,
             "V": wordnet.VERB,
             "R": wordnet.ADV
             }
    return word_pos.get(pos,wordnet.NOUN)
arr1=[]
obj=WordNetLemmatizer()
for i in arr:
    arr1.append([obj.lemmatize(w,get_wordnet_pos(w)) for w in i])


   


In [ ]:
import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()
arr3=[]
arr4=[]
for array in arr:
    for w in array:
        arr3.append(ps.stem(w))
    arr4.append(arr3) 

In [ ]:
#Word embedding

from sklearn.feature_extraction.text import TfidfVectorizer
vecterize=TfidfVectorizer()
vecterize.fit(df_train_list)

vector1=vecterize.transform(df_train_list)




In [ ]:
type(vector1)

In [ ]:
vector=np.array(vector1)

In [ ]:
vector2=vector1.toarray()

In [ ]:
type(vector)
vector

In [ ]:
Y_train_s.shape

In [ ]:
#Initializing Support Vector Machine and fitting the training data

from sklearn.svm import SVC
classifier = SVC(kernel='rbf', random_state = 1)
classifier.fit(vector1,Y_train_s)

In [ ]:
#Predicting the classes for test set

Y_pred = classifier.predict(X_test)

In [ ]:
#Attaching the predictions to test set for comparing

test_set["Predictions"] = Y_pred

In [ ]:
#Calculating the accuracy of the predictions

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test_s,Y_pred)
accuracy = float(cm.diagonal().sum())/len(Y_test_s)
print("\nAccuracy Of SVM For The Given Dataset : ", accuracy)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:

# define training data arr1
# train model
model = Word2Vec(token_train, min_count=1)

#This is vocabulary of on which the model is train
words1=list(model.wv.vocab)
#choose a arbitary word from the vocabulary
print(model['girl'].shape)





In [ ]:
type(model)

In [ ]:
#now we do dimensionality reduction on our train model
#fit 2d pca model to vector

# summarize vocabulary

words = list(model.wv.vocab)
# print(new_model)
# fit a 2d PCA model to the vectors
X = model[model.wv.vocab]
pca = PCA(n_components=1)
result = pca.fit_transform(X)



In [ ]:
type(X)

In [ ]:
print(result)
print(type(result))
from itertools import chain 
result_list = list(chain.from_iterable(list(result))) 


In [ ]:
result_list

In [ ]:
Dict=list(model.wv.vocab)
print(Dict)


In [ ]:
# create a scatter plot of the projection
plt.figure(figsize= (20,20))
plt.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)
for i, word in enumerate(words):
    plt.annotate(word, xy=(result[i, 0], result[i, 1]))
plt.show()

In [ ]:
    w1='male'
    w2='female'
    w3='queen'
    r = model.most_similar(positive=[w1, w3], negative=[w2])
    print("%s - %s = %s - %s" % (w1, w2, r[0][0], w3))


In [ ]:
print(type(model['dog']))

In [ ]:
vec_word=list(model.wv.vocab)

Word_dic={k:v for k, v in zip(vec_word,result_list)}


In [ ]:
Word_dic

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE()


num_of_words = 200


Z = tsne.fit_transform(X1[:num_of_words])
plt.figure(figsize= (20,20))
plt.scatter(Z[:,0], Z[:,1])
for i in range(num_of_words):
        try:
            plt.annotate(vec_word[i, xy=(Z[i,0], Z[i,1])
        except:
            print("bad string:", X1[i])
plt.draw()

In [ ]:
print('Loading word vectors...')
word2vec = {}
embedding = []
idx2word = []
for line in model:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    word2vec[word] = vec
    embedding.append(vec)
    idx2word.append(word)
print('Found %s word vectors.' % len(word2vec))